In [1]:
import numpy as np
import json
import matplotlib.pyplot as plt

In [2]:
# load dataset

In [3]:
# load model
# predict loal scene coords

In [4]:
# unwhiten local scene coords -> get world scene coords

In [5]:
# save error images into a new directory
# input image - error map

In [6]:
# get all predictions into a sinle ply file

In [ ]:
# inspect where the model makes errors
#   Q: what object categories does the model poorly predict on?
#   Q: what parts of the building does it poorly predict on?
#   Q: which cameras does the model predict worse on?
#   Q: how does positioning of validation cameras affects error?
#   Q: 